# Utilities for Thesis
### Figure generators and such

# Generic Imports

In [ ]:
import numpy as np
import madmom

# Figures

### Time Domain Audio Signal Representation

### Frequency Domain Audio Signal Representation

### Time/Frequency Domain Audio Signal Representation

### Mel Scale

### Decibel Scale